### Eigen Value Decomposition (EVD) vs Singular Value Decomposition (SVD) from Scratch and with Sklearn

##### Case 1: $ X $ is $ n \times m $ (features × samples)
Here, $ X $ is the **centered** data matrix and has **$ n $ features** and **$ m $ training samples**.

**Eigenvalue Decomposition (EVD)**
1. Compute the covariance matrix:
   $$
   \hat{\Sigma} = \frac{1}{m} X X^T
   $$
   where $ \hat{\Sigma} $ is **$ n \times n $**.
2. Eigenvalue decomposition:
     $$
     \hat{\Sigma} = U \Lambda U^T
     $$
     where:
     - $ U $ is **$ n \times n $** (eigenvectors).
     - $ \Lambda $ is **$ n \times n $** (diagonal matrix of eigenvalues).
3. Reordering:  
   - The eigenvalues in $ \Lambda $ must be sorted in **descending** order.  
   - The corresponding eigenvectors in $ U $ must be **rearranged accordingly**.  
4. Selection of principal components:  
   - Choose the first $ k $ eigenvectors from the reordered $ U $, forming $ U_{\text{reduce}} $ (which is $ n \times k $).  
5. Projection:
     $$
     Z = U_{\text{reduce}}^T X
     $$
   where $ U_{\text{reduce}}^T $ is $ k \times n $ and $ X $ is $ n \times m $ ... therefore $ Z $ is $ k \times m $.
   

**Singular Value Decomposition (SVD)**
1. SVD of $ X $:
   $$
   X = U S V^T
   $$
   where:
   - $ U $ is **$ n \times n $** (left singular vectors).
   - $ S $ is **$ n \times m $** (diagonal matrix of singular values).
   - $ V $ is **$ m \times m $** (right singular vectors).
2. Selection of principal components:  
   - Choose the first $ k $ columns of $ U $, forming $ U_{\text{reduce}} $ (which is $ n \times k $).
   - Note that here eignevalues and corresponding eigenvectors are already oredered.  
3. Projection:
   $$
   Z = U_{\text{reduce}}^T X
   $$
   where $ U_{\text{reduce}}^T $ is $ k \times n $ and $ X $ is $ n \times m $ ... therefore $ Z $ is $ k \times m $.

---

##### Case 2: $ X $ is $ m \times n $ (samples × features)
Here, $ X $ is the **centered** data matrix and has **$ m $ samples** and **$ n $ features**.

**Eigenvalue Decomposition (EVD)**
1. Compute the covariance matrix:
   $$
   \hat{\Sigma} = \frac{1}{m} X^T X
   $$
   where $ \hat{\Sigma} $ is **$ n \times n $**.
2. Eigenvalue decomposition:
   $$
   \hat{\Sigma} = U \Lambda U^T
   $$
   where:
   - $ U $ is **$ n \times n $** (eigenvectors).
   - $ \Lambda $ is **$ n \times n $** (diagonal matrix of eigenvalues).
3. Reordering:  
   - The eigenvalues in $ \Lambda $ must be sorted in **descending** order.  
   - The corresponding eigenvectors in $ U $ must be **rearranged accordingly**.  
4. Selection of principal components:  
   - Choose the first $ k $ eigenvectors from the reordered $ U $, forming $ U_{\text{reduce}} $.  
5. Projection:
   $$
   Z = X U_{\text{reduce}}
   $$
   where $ X $ is $ m \times n $ and $ U_{\text{reduce}} $ is $ n \times k $ ... therefore $ Z $ is $ m \times k $.

**Singular Value Decomposition (SVD)**
1. SVD of $ X $:
   $$
   X = U S V^T
   $$
   where:
   - $ U $ is **$ m \times m $** (left singular vectors).
   - $ S $ is **$ m \times n $** (diagonal matrix of singular values).
   - $ V $ is **$ n \times n $** (right singular vectors).
2. Selection of principal components:  
   - Choose the first $ k $ columns of $ V $, forming $ V_{\text{reduce}} $.  
3. Projection:
   $$
   Z = X V_{\text{reduce}}
   $$
   where $ X $ is $ m \times n $ and $ V_{\text{reduce}} $ is $ n \times k $ ... therefore $ Z $ is $ m \times k $.

In [1]:
import numpy as np
import pandas as pd
from collections import Counter

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

In [12]:
df = pd.read_csv('../datasets/glass.csv')
features = df.columns[:-1].tolist()

display(df)

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0,1
...,...,...,...,...,...,...,...,...,...,...
209,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0,7
210,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0,7
211,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0,7
212,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0,7


In [13]:
# Remove outliers from the dataset
def outlier_hunt(df):
    outlier_indices = []

    for col in df.columns.tolist():
        Q1 = np.percentile(df[col], 25)
        Q3 = np.percentile(df[col], 75)
        IRQ = Q3 - Q1
        outlier_step = 1.5 * IRQ

        outlier_list_col = df[(df[col] < Q1 - outlier_step) |
                              (df[col] > Q3 + outlier_step)].index
        outlier_indices.extend(outlier_list_col)

    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(k for k, v in outlier_indices.items() if v > 2)

    return multiple_outliers

print(df.shape)

(214, 10)


In [14]:
# Remove outliers from the dataset
outlier_indices = outlier_hunt(df[features])
df = df.drop(outlier_indices).reset_index(drop=True)
print(df.shape)

(200, 10)


In [38]:
X = df[features]
y = df['Type']

test_size = 0.2
seed = 42

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

Eigen Value Decomposition (EVD)

In [39]:
# 1. Center the data (zero mean) or Normalize them
sc = StandardScaler()
X_train_normalized = sc.fit_transform(X_train)
X_test_normalized = sc.transform(X_test)

In [40]:
# 2. Compute the covariance matrix
cov_matrix = np.cov(X_train_normalized, rowvar=False) # rowvar=False to indicate that each row is an observationa and each column a feature

In [41]:
# 3. Perform eigenvalue decomposition
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

In [42]:
print("Size of X_train_normalized: ", X_train_normalized.shape)
print("Size of Covariance Matrix: ", cov_matrix.shape)
print("Size of U: ", eigenvectors.shape)
print("Size of main diagonal elements of Λ: ", eigenvalues.shape)

Size of X_train_normalized:  (160, 9)
Size of Covariance Matrix:  (9, 9)
Size of U:  (9, 9)
Size of main diagonal elements of Λ:  (9,)


In [43]:
print(eigenvalues)

[3.05122971e+00 2.46846798e+00 1.28161063e+00 9.61418033e-01
 5.57542845e-01 2.57775763e-03 8.98503006e-02 3.89509520e-01
 2.54396998e-01]


In [44]:
print(eigenvectors)

[[ 1.47053409e-01  5.71310135e-01 -1.79644456e-01  6.62484608e-03
   6.11620910e-02 -2.62474267e-02  7.01871715e-01 -3.40522400e-01
   8.15773679e-02]
 [-3.94713964e-01  2.60398801e-01  4.34721069e-01  1.81554514e-02
   9.85527482e-03  3.28388111e-01  9.99907144e-02  5.64658644e-02
  -6.82611828e-01]
 [ 4.92910732e-01 -4.66807448e-02  4.17354296e-01  9.19528469e-02
  -6.83757574e-02  6.85876295e-01  5.35316095e-02  3.44277663e-02
   3.05069734e-01]
 [-4.19491094e-01 -2.43528848e-01 -3.80171782e-02 -9.39234016e-02
   6.80566348e-01  2.17851314e-01  3.07531519e-01  2.40283079e-01
   3.02510610e-01]
 [-1.71040935e-01 -4.55759514e-01 -4.04697310e-01 -8.33459104e-04
  -5.53794254e-01  3.01870473e-01  4.21148774e-01  9.00228524e-02
  -1.26346331e-01]
 [ 3.28813123e-01 -3.68930158e-01 -2.40055474e-01  5.41610927e-02
   4.31390728e-01  1.53934530e-01 -6.37597851e-02 -5.24012825e-01
  -4.54712946e-01]
 [-4.12105010e-02  4.47489345e-01 -6.07588042e-01 -4.85188142e-02
   6.73185990e-02  4.7038515

In [45]:
# 4. Sort eigenvalues and eigenvectors in descending order
sorted_indices = np.argsort(eigenvalues)[::-1]
eigenvalues_ordered = eigenvalues[sorted_indices]
eigenvectors_ordered = eigenvectors[:, sorted_indices]

In [46]:
print(eigenvalues_ordered)

[3.05122971e+00 2.46846798e+00 1.28161063e+00 9.61418033e-01
 5.57542845e-01 3.89509520e-01 2.54396998e-01 8.98503006e-02
 2.57775763e-03]


In [47]:
# 5. Project the data onto the top k principal components
k = 4
U_reduced = eigenvectors_ordered[:, :k]

evd_projected_data = X_test_normalized @ U_reduced # @ is equivalent to the dot product

In [48]:
print("Size of X_test_normalized: ", X_test_normalized.shape)
print("Size of U_reduced: ", U_reduced.shape)
print("Size of Z: ", X_test_normalized.shape, " × ", U_reduced.shape, " = ", evd_projected_data.shape)

Size of X_test_normalized:  (40, 9)
Size of U_reduced:  (9, 4)
Size of Z:  (40, 9)  ×  (9, 4)  =  (40, 4)


In [49]:
print(evd_projected_data)

[[ 0.59762705  0.22750794  0.43090147  0.69106272]
 [ 0.88940714 -1.00208547 -0.31228539  0.74495165]
 [ 1.21866925 -0.88091848 -0.44850229 -0.78265381]
 [ 0.80977144  1.31023317 -3.5895215   0.36927705]
 [ 0.79419679 -0.43613607  0.57220814  0.78060014]
 [ 0.89488731 -1.49672785  0.30135602 -1.55728252]
 [ 1.18218844  3.06110549 -0.39270715  0.37028767]
 [-1.1521304   1.05037318  0.15695239  0.45248883]
 [-1.39195044  2.53324274 -1.96500117  0.45246464]
 [ 0.75059762  0.78424382  0.40456783  0.70219683]
 [ 1.46775639  2.13677084 -0.3211067  -1.11827915]
 [-3.89417864 -0.93939153 -0.48556002 -0.58654539]
 [ 0.45353538  0.41462323 -1.14330158 -5.13491364]
 [ 0.27133996 -0.71463744  1.25125414 -0.7489306 ]
 [-4.09440067 -0.19157319  0.35757089  0.03810096]
 [-2.49393336 -0.32595273  0.42437212  0.21731024]
 [ 1.13462413 -2.19849983  0.57551199 -2.56062902]
 [ 0.47931589  0.19191638  0.550641    0.67527606]
 [ 0.69443737 -1.03792743  0.534929    0.78644731]
 [ 1.40747979  2.10285016 -0.30

Singular Value Decomposition (SVD)

In [51]:
# Perform Singular Value Decomposition
U, S, Vt = np.linalg.svd(X_train_normalized)

print("Size of U: ", U.shape)
print("Size of main diagonal elements of S: ", S.shape)
print("Size of V^T: ", Vt.shape)

Size of U:  (160, 160)
Size of main diagonal elements of S:  (9,)
Size of V^T:  (9, 9)


In [52]:
# Project data onto the top k principal components
k = 4
V_reduced = Vt[:k, :].T  # Top k left singular vectors

svd_projected_data = X_test_normalized @ V_reduced

In [53]:
print("Size of X_test_normalized: ", X_test_normalized.shape)
print("Size of V_reduced: ", V_reduced.shape)
print("Size of Z: ", X_test_normalized.shape, " × ", V_reduced.shape, " = ", svd_projected_data.shape)

Size of X_test_normalized:  (40, 9)
Size of V_reduced:  (9, 4)
Size of Z:  (40, 9)  ×  (9, 4)  =  (40, 4)


In [55]:
print(svd_projected_data)

[[ 0.59762705 -0.22750794 -0.43090147 -0.69106272]
 [ 0.88940714  1.00208547  0.31228539 -0.74495165]
 [ 1.21866925  0.88091848  0.44850229  0.78265381]
 [ 0.80977144 -1.31023317  3.5895215  -0.36927705]
 [ 0.79419679  0.43613607 -0.57220814 -0.78060014]
 [ 0.89488731  1.49672785 -0.30135602  1.55728252]
 [ 1.18218844 -3.06110549  0.39270715 -0.37028767]
 [-1.1521304  -1.05037318 -0.15695239 -0.45248883]
 [-1.39195044 -2.53324274  1.96500117 -0.45246464]
 [ 0.75059762 -0.78424382 -0.40456783 -0.70219683]
 [ 1.46775639 -2.13677084  0.3211067   1.11827915]
 [-3.89417864  0.93939153  0.48556002  0.58654539]
 [ 0.45353538 -0.41462323  1.14330158  5.13491364]
 [ 0.27133996  0.71463744 -1.25125414  0.7489306 ]
 [-4.09440067  0.19157319 -0.35757089 -0.03810096]
 [-2.49393336  0.32595273 -0.42437212 -0.21731024]
 [ 1.13462413  2.19849983 -0.57551199  2.56062902]
 [ 0.47931589 -0.19191638 -0.550641   -0.67527606]
 [ 0.69443737  1.03792743 -0.534929   -0.78644731]
 [ 1.40747979 -2.10285016  0.30

Sklearn SVD

In [56]:
pca = PCA(n_components=4, random_state=seed) # SVD
# The 'fit_transform' method first performs the Singular Value Decomposition (SVD) and then projects the original data
# into the lower k-dimensional space by multipling the original data onto these components (U_reduced).
_ = pca.fit_transform(X_train_normalized)
# Transform the test data into the same k-dimensional space using the components learned from the training set.
sklearn_svd_projected_data = pca.transform(X_test_normalized)

print(sklearn_svd_projected_data.shape)
print(sklearn_svd_projected_data)

(40, 4)
[[ 0.59762705  0.22750794 -0.43090147 -0.69106272]
 [ 0.88940714 -1.00208547  0.31228539 -0.74495165]
 [ 1.21866925 -0.88091848  0.44850229  0.78265381]
 [ 0.80977144  1.31023317  3.5895215  -0.36927705]
 [ 0.79419679 -0.43613607 -0.57220814 -0.78060014]
 [ 0.89488731 -1.49672785 -0.30135602  1.55728252]
 [ 1.18218844  3.06110549  0.39270715 -0.37028767]
 [-1.1521304   1.05037318 -0.15695239 -0.45248883]
 [-1.39195044  2.53324274  1.96500117 -0.45246464]
 [ 0.75059762  0.78424382 -0.40456783 -0.70219683]
 [ 1.46775639  2.13677084  0.3211067   1.11827915]
 [-3.89417864 -0.93939153  0.48556002  0.58654539]
 [ 0.45353538  0.41462323  1.14330158  5.13491364]
 [ 0.27133996 -0.71463744 -1.25125414  0.7489306 ]
 [-4.09440067 -0.19157319 -0.35757089 -0.03810096]
 [-2.49393336 -0.32595273 -0.42437212 -0.21731024]
 [ 1.13462413 -2.19849983 -0.57551199  2.56062902]
 [ 0.47931589  0.19191638 -0.550641   -0.67527606]
 [ 0.69443737 -1.03792743 -0.534929   -0.78644731]
 [ 1.40747979  2.102850